<a href="https://colab.research.google.com/github/HueyVault/study_NLPs/blob/main/quests/DeepLearnings/linearregression_house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 수집


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/finance_AI/datasets/house-prices-advanced-regression-techniques_train.csv'
data_df = pd.read_csv(file_path)
data_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [26]:
target_column = 'LotArea'
feature_columns = ['BsmtUnfSF','LowQualFinSF','HalfBath','Fireplaces',target_column]

select_df = data_df[feature_columns]
select_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   BsmtUnfSF     1460 non-null   int64
 1   LowQualFinSF  1460 non-null   int64
 2   HalfBath      1460 non-null   int64
 3   Fireplaces    1460 non-null   int64
 4   LotArea       1460 non-null   int64
dtypes: int64(5)
memory usage: 57.2 KB



## 데이터 전처리
- 데이터 분석

In [9]:
from sklearn.preprocessing import StandardScaler


In [27]:
standardscaler = StandardScaler()
standardscaler.fit(select_df)

StandardScaler()

In [30]:
scailing_array = standardscaler.transform(select_df)


In [31]:
scailing_array.shape, type(scailing_array)

((1460, 5), numpy.ndarray)

In [34]:
scailing_array[:5,:4], scailing_array[:5,4]

(array([[-0.94459061, -0.12024172,  1.22758538, -0.95122649],
        [-0.64122799, -0.12024172, -0.76162067,  0.60049493],
        [-0.30164298, -0.12024172,  1.22758538,  0.60049493],
        [-0.06166957, -0.12024172, -0.76162067,  0.60049493],
        [-0.17486457, -0.12024172,  1.22758538,  0.60049493]]),
 array([-0.20714171, -0.09188637,  0.07347998, -0.09689747,  0.37514829]))

## 데이터 분할
- train, test, validation

In [13]:
from sklearn.model_selection import train_test_split


In [35]:
train_features, test_features, train_label, test_label =\
      train_test_split(scailing_array[:,:4], scailing_array[:,4], test_size=0.2, random_state=42)
train_features.shape, test_features.shape, train_label.shape, test_label.shape

((1168, 4), (292, 4), (1168,), (292,))

In [16]:
import torch

In [36]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_label_tensor = torch.tensor(train_label, dtype=torch.float32).view(-1,1) # 행만 있는 경우 행열로 변환 해줌
# test_features_tensor = torch.tensor(test_features)
# test_label_tensor = torch.tensor(test_label)
type(train_features_tensor), type(train_label_tensor)

(torch.Tensor, torch.Tensor)

In [37]:
train_features_tensor.shape, train_label_tensor.shape

(torch.Size([1168, 4]), torch.Size([1168, 1]))

## 모델 학습



In [38]:
# simple model linear regression
# model, loss function, optimizer function
class LinearRegression(torch.nn.Module) :
    def __init__(self, input_dim, output_dim): # input : feature 의 열 갯수, output : label의 카테고리 갯수 (연속형은 값 하나)
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim,output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out


In [39]:
train_features_tensor.shape[1], train_label_tensor.shape[1]

(4, 1)

In [40]:
model = LinearRegression(train_features_tensor.shape[1], train_label_tensor.shape[1])

In [41]:
model

LinearRegression(
  (linear): Linear(in_features=4, out_features=1, bias=True)
)

In [42]:
criterion = torch.nn.MSELoss() # Loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # Optimizer fuction, lr = learnning rate : gradient decsent 빠르게 찾기 최근에 ADAMW 가 제일 좋다. 찾아가는 과정 어렵다. 그래서 optimizer 성능 중요하다.

In [43]:
## 반복 학습
# for epoch in range(10):
for epoch in range(1000):
    pred_y = model.forward(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"epoch : {epoch}, loss : {loss.item()}")



epoch : 0, loss : 1.5505590438842773
epoch : 100, loss : 1.0873231887817383
epoch : 200, loss : 1.0740430355072021
epoch : 300, loss : 1.073575735092163
epoch : 400, loss : 1.0735571384429932
epoch : 500, loss : 1.0735563039779663
epoch : 600, loss : 1.0735563039779663
epoch : 700, loss : 1.0735563039779663
epoch : 800, loss : 1.0735563039779663
epoch : 900, loss : 1.0735563039779663


## 모델 평가



In [44]:
model.eval()

LinearRegression(
  (linear): Linear(in_features=4, out_features=1, bias=True)
)

In [45]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor) # 예측도 간은 loss function
    print(f"loss : {loss.item()}")


loss : 1.0735563039779663


## 모델 배포